<a href="https://colab.research.google.com/github/TheNobody-12/Chat-Toxicity-Analyser/blob/main/TopicModelingUsingBertTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [105]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

class TextPreprocessing:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def remove_emoji(self, string):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', string)

    def remove_urls(self, text):
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        return url_pattern.sub(r'', text)

    def remove_numbers(self, text):
        text = re.sub(r'\d+', '', text)
        return text

    def preprocess_text(self, text):
        # Converting Object Data type into String Datatype
        # text = str(text)
        # Removal of Punctuation - !"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`
        text = re.sub(r'[^\w\s]', '', text)
        # Remove URLs
        text = self.remove_urls(text)
        # Removing Emojis
        text = self.remove_emoji(text)
        # Remove numbers
        text = self.remove_numbers(text)
        # Lower casing the Text
        text = text.lower()
        # Stopwords removing
        text = [word for word in word_tokenize(text) if word not in self.stop_words]
        text = " ".join(text)
        # Tokenization
        text = word_tokenize(text)
        # Stemming and Lemmatization
        text = [self.lemmatizer.lemmatize(word) for word in text]
        return text


# Recommendation Engine

In [23]:
import pandas as pd
import numpy as np

In [147]:
import pandas as pd

post_text =  [
        " Exciting news! I just discovered 5 valuable tips that can significantly improve your coding skills. These practical techniques will help you write cleaner and more efficient code, boost your productivity, and become a better programmer. Check them out now! #CodingTips #ProgrammingSkills",
        " Artificial Intelligence is reshaping the world as we know it! In my latest post, I delve into the future of AI and its potential impact on various industries. From self-driving cars to personalized healthcare, AI is revolutionizing our lives. Join the conversation and share your thoughts! #ArtificialIntelligence #FutureTech",
        " Are you interested in investing in cryptocurrency? Look no further! I've put together a comprehensive guide on how to get started with cryptocurrency investments. Learn about different cryptocurrencies, wallets, and strategies to make informed investment decisions. Let's dive into the exciting world of crypto! #Cryptocurrency #InvestmentTips",
        " Building a strong personal brand is key to career success. In my recent blog post, I share practical tips on how to craft a compelling personal brand that stands out in today's competitive job market. From defining your unique value proposition to leveraging social media, discover strategies to enhance your professional reputation! #PersonalBranding #CareerAdvice",
        " As an entrepreneur, productivity is crucial to success. In this post, I share my top 10 must-have productivity apps that can supercharge your efficiency and streamline your workflow. From project management to time tracking, these apps will help you stay organized and focused. Get ready to boost your productivity! #ProductivityHacks #EntrepreneurLife",
        " The benefits of remote work are undeniable. In my latest article, I discuss the advantages of working remotely, including flexibility, improved work-life balance, and access to a global talent pool. Whether you're a remote worker or considering remote opportunities, this post is a must-read! #RemoteWork #WorkFromAnywhere",
        " Financial planning is essential, especially for young professionals. In this comprehensive guide, I cover key aspects of financial planning, such as budgeting, saving, investing, and debt management. Take control of your financial future and make informed decisions. Start building your wealth today! #FinancialPlanning #MoneyManagement",
        " Wondering which programming languages will be in demand in 2023? Look no further! I've curated a list of the top programming languages that will help you stay ahead in the rapidly evolving tech industry. Whether you're a beginner or an experienced developer, this post will guide you in choosing the right language to master. #ProgrammingLanguages #TechTrends",
        " Attention aspiring data scientists! I've compiled a list of 10 essential skills you need to succeed in the field of data science. From programming languages to statistical analysis and machine learning, these skills will give you a competitive edge in this high-demand career. Start honing your data science skills today! #DataScience #CareerAdvice",
        " Dreaming of starting your own business? In this post, I share practical tips and strategies to help you navigate the entrepreneurial journey. From validating your business idea to creating a business plan and finding funding, this guide has got you covered. Turn your dreams into reality! #Entrepreneurship #StartupTips",
        " Blockchain technology is revolutionizing industries across the globe. In my recent blog post, I explore the impact of blockchain on sectors like finance, supply chain, healthcare, and more. Discover how this decentralized technology is transforming traditional systems and opening up new possibilities. #Blockchain #TechInnovation",
        " Saving money and building wealth go hand in hand. In this post, I share practical tips on how to save effectively, invest wisely, and achieve long-term financial goals. Whether you're just starting your financial journey or looking to optimize your savings strategy, this post has valuable insights for you! #FinancialTips #WealthBuilding",
        " Public speaking is an art that can be mastered! In this post, I reveal secrets to becoming a confident and captivating public speaker. From overcoming stage fright to structuring your speech and engaging the audience, these tips will help you deliver impactful presentations. Unleash your inner speaker! #PublicSpeaking #CommunicationSkills",
        " E-commerce is booming in the digital age. In my latest article, I explore the rise of online shopping, the benefits it offers to businesses and consumers, and key trends shaping the e-commerce landscape. Discover how e-commerce is transforming the way we shop and do business. #EcommerceTrends #DigitalBusiness",
        " Networking is a powerful tool for career growth. In this post, I share the importance of building a strong professional network and practical tips on how to expand your network effectively. From attending events to leveraging social media, discover strategies to enhance your networking game! #NetworkingTips #CareerDevelopment",
        " Cybersecurity is everyone's responsibility. In this post, I outline best practices for individuals to protect their online privacy and secure their digital lives. From strong passwords to safe browsing habits and data encryption, these tips will help you stay safe in the digital world. Don't compromise on security! #Cybersecurity #OnlineSafety",
        " Want to ace your next job interview? Look no further! I've compiled a comprehensive guide with expert tips and strategies to help you prepare for interviews, answer common interview questions, and make a lasting impression on recruiters. Get ready to land your dream job! #JobInterview #CareerSuccess",
        " Investing in stocks or real estate? It's a common dilemma for many investors. In this post, I weigh the pros and cons of each investment option, discuss the risks and potential returns, and provide insights to help you make an informed decision. Choose the right investment path for your financial goals! #InvestmentOptions #FinancialDecisions",
        " Effective project management is crucial for successful outcomes. In this post, I share 10 essential tools that can streamline your project management process, enhance collaboration, and ensure project success. From task management to communication platforms, these tools will supercharge your productivity. #ProjectManagement #ProductivityTools",
        " Big Data is transforming the business landscape. In this post, I explore the role of big data in decision-making, customer insights, and operational efficiency. Discover how businesses can leverage big data analytics to gain a competitive edge and make data-driven decisions. Stay ahead of the curve! #BigData #BusinessAnalytics",
        " Looking to create engaging social media content? You're in luck! In this post, I share practical tips and strategies to craft compelling content that captivates your audience, increases engagement, and boosts your brand's online presence. Unlock the secrets of social media success! #SocialMediaMarketing #ContentCreation",
        " Financial independence and early retirement are within reach. In this post, I discuss the principles of financial independence, share strategies to achieve financial freedom, and outline steps to retire early. Take control of your finances and design the life you desire! #FinancialIndependence #RetireEarly",
        " The evolution of mobile technology has transformed the way we live and work. In this post, I take a journey through the history of mobile technology, explore the latest advancements like 5G and foldable phones, and discuss the future possibilities of this rapidly evolving field. Stay connected to the future! #MobileTech #TechEvolution",
        " Navigating the job market in the digital era can be challenging. In this post, I share insights on how to stand out in a competitive job market, leverage online platforms for job search, and develop the skills needed for the jobs of the future. Unlock your career potential! #JobSearch #CareerDevelopment",
        " The future of work is intertwined with automation and AI. In this post, I discuss the impact of automation on jobs, the rise of AI-powered technologies, and the skills needed to thrive in the future workplace. Get ready to embrace the changing landscape of work! #FutureOfWork #Automation",
        "Calling all millennials! Financial success starts with smart money management. In this post, I share 10 personal finance tips specifically tailored to the needs of millennials. From budgeting to investing and managing student loans, these tips will set you on the path to financial freedom. Take control of your financial future! #MillennialFinance #MoneyTips",
        "Reflecting on an incredible journey of innovation and collaboration at the Intel® oneAPI hackathon finale! From the inspiring inaugural session to Sachin sir's insightful opening note, we've come a long way. Now, it's time to unleash our creativity and compete for the ultimate victory. Brace yourselves for an unforgettable finale! Intel Software Kavita Aroor Hack2skill Simarjit Mukherjee Shakti Singh Abhishek Nandy @intelindia #oneAPI #Intelhackathon #hack2skillIndian Institute of Science (IISc)",
        "The Machine Learning Bootcamp held from 19-28th June, centered around the roadmap to mastering machine learning algorithms, was an astounding success! From decoding complex algorithms to practical implementations, participants dove into the world of machine learning, honing their skills over the course of the program.We successfully trained a significant number of participants, providing them with a solid foundation in machine learning and empowering them to build robust algorithms. We are thrilled to have played a part in your journey towards becoming skilled machine learning practitioners.Stay tuned for more exciting updates and resources as we continue to support your growth in the field of machine learning. Keep pushing the boundaries and exploring new horizons! 🌟#kiet_group_of_institutions #DSDLClub #MachineLearningBootcamp #AlgorithmMastery #machinelearning",
        "From 'One Day' to 'Day-1'...It was back in 2015 when I had decided to take my 1st attempt at the CAT in 2016 with an aspiration to get into the best management institute in the country.Over the years I continued to take the CAT multiple times along with my consulting job, in pursuit of this aspiration.Even though I had chances of joining an MBA programme at several B-Schools in the past years and I'm grateful for all the opportunities, my conscience just didn't allow me to settle for less than what I had always dreamt of.Fast-forward to CAT 2022, I scored 99.39%ile and received calls for the interview process from multiple top IIMs/B-Schools in the country.I am elated and humbled to announce that I've finally started my MBA journey at my dream place, the Indian Institute of Management Ahmedabad (IIM Ahmedabad PGP 2023-25).I am also humbled and grateful to have received admission offers from other top institutes -Indian Institute of Management Bangalore (PGP and PGP-BA), Indian Institute of Management, Lucknow, XLRI Jamshedpur (GMP), BITSoM - BITS School of Management (along with 100% merit scholarship), Indian Institute of Management, Indore, Indian Institute of Management, Shillong, among other institutes.One key learning which I'd like to share from my experience:Never give up on your dreams!It shouldn't matter to you if others believe in your dream/aim or not; if you do, you should absolutely go for it!Everyone takes their own sweet time and move at their own pace in achieving things and it's fine if it's taking you longer too, you should just not give up on them. Ultimately it's better to keep trying and then achieving them/seeing how things pan out, than later regretting that you could've tried and not knowing what would have happened#IIMAhmedabad #IIMA #WIMWI #MBA #PGP #CAT #CAT2022 #Bain #KPMG #ManagementConsulting",
        """
        Hi there!
        So far in our high-income skills series, we’ve explored how you can use data analysis to demonstrate your impact and project management to enhance your workflow. Both of these skill sets tend to involve internal workflows. This week, we’re shifting to an external focus: the user experience.
        User experience (UX) has to do with understanding the way a person (the user) interacts with (or experiences) a product. The product can be a physical item, such as a sneaker, or a digital asset, such as a web page. When you consider a user’s experience, you’re making your product more user-friendly, oftentimes by drawing upon your creativity and social perceptiveness.
        What can you do with strong UX skills?
        Typically, when we think about UX, jobs like UX designer or UX researcher may come to mind. People in these roles typically work with products that are crucial to a business’s success. At Coursera, for example, UX designers help design the online classroom experience.
        But we can also broaden that perspective: Anyone can be considered a user, and anything that a user interacts with can be considered a product. Say you’re building a portfolio to share with potential employers. Your potential employers would be your users, and your portfolio would be your product. Tapping into your UX skills, you may consider the type of experience you want your potential employer to have when they look at your portfolio, and then use design elements that encourage that experience.
        Here are some ways you can incorporate UX skills into your regular workflow:
        1. Conduct research to understand your target customer.
        Knowing your audience is a crucial aspect of effective communication. People skilled in UX often gather data about their target audience in order to determine how to develop a product that speaks to them.
        Use this two-hour Guided Project to practice using Google Forms to analyze user research data like a pro, or learn more about UX research with the University of Michigan’s User Experience Research and Design Specialization.
        2. Design attractive and engaging products.
        Whether you’re writing a data report or making a presentation to the board, good technical design can be the difference between a captivated audience and a bored one. Next time you’re tasked with sharing your latest project, impress your colleagues and supervisors with a top-tier visual experience.
        Boost your design capabilities with the University of Colorado’s Graphic Design Elements for Non-Designers Specialization.
        3. Connect your customers’ needs with your business goals.
        Design thinking involves more than just compelling visuals; it’s an iterative problem-solving process. In this way, you can use UX and design principles to solve complex business problems and help move closer to your team’s goals.
        Learn more about design thinking with Innovation Through Design: Think, Make, Break, Repeat, a free course from the University of Sydney.
        Where to begin
        If you’re new to user experience, start with the first course in Google’s UX Design Professional Certificate, Foundations of User Experience (UX) Design. This introductory course covers the basics of design thinking, process, and research, and if you enjoy it, you can advance to the next course in the certificate program.
        For a crash course, try Georgia Tech’s Introduction to User Experience Design. This single course runs through the design process in about six hours.
        To enhance your web development skills, consider CalArts’ UI / UX Design Specialization. Here, you’ll learn how to incorporate design principles into your web design.
        That’s all for this week. Next week, we’ll talk about artificial intelligence skills. See you then!
        """
        ,""" Hi again!
        We are back with the second issue of our high-income skills series.
        Last week, we discussed how you can benefit from data analysis skills, even if you aren’t pursuing a career as a data analyst. In the comment section, Cathyann Martin emphasized the link between data analysis and good communication, saying that “being able to share the findings from the data in non-technical terms, so that others can understand and use it to make decisions, is important.”
        This is a great point; it highlights how your technical skills and workplace skills can interact with and enhance one another. Project management, our topic for this week, is another technical skill that is boosted by several workplace skills.
        Project managers incorporate many tools and processes into their workflow, but their ultimate goal is to get things done, and they do this with strong organization, persuasion, negotiation, and risk management skills. For a practical explanation of this skill set, watch the first lecture video of the Google Project Management: Professional Certificate.
        If you have strong project management skills, then you're a planner.
        You see the big picture and are able to identify the tasks and people that will enable you to achieve your desired outcome. You strategically navigate roadblocks while keeping everyone involved happy. Most importantly, you are a finisher. If you commit to doing something, you will get it done or find a reasonable alternative.
        Here are some ways you can (and maybe already do) flex your project management muscles:
        1. Concentrate your organizational efforts into one place.
        Perhaps you use a calendar or to-do lists to stay on track. Take your organization a step further by consolidating your thoughts, tasks, and deadlines into a single, shareable source of information, such as a spreadsheet. This exercise showcases your ability to manage goals and collaborate.
        Get a sense for how project managers organize their efforts with Guided Projects that help you create a project management tracker and create a simple Gantt chart. Both projects take about two hours to complete and will leave you with a reusable template for your future planning needs.
        2. Practice managing projects in your everyday life.
        Managing the needs of others, effectively communicating, and making decisions are key project management concepts, and they also tend to show up in daily life. Next time you’re asked to make a decision, tune into your inner project manager.
        For example, when you’re asked “What do you want to eat?” consider:
        Risk management: How much longer can the vegetables in your refrigerator last? Prioritize accordingly or run the risk of wilted produce.
        Budgeting: Want to eat out for under $15 per person? You can rule out everything that exceeds that budget.
        Communication: Share your selection with the person who posed the question. Offer your reasoning to get them on board with your decision (also called stakeholder alignment) and draw upon your negotiation skills as needed.
        Where to begin
        If you’re new to project management, start with the first course in the Google Project Management: Professional Certificate, Foundations of Project Management. Here, you’ll get an overview of the skill set, and if you enjoy the course, you can continue on to earn a certificate.
        For a crash course, check out Fundamentals of Project Planning and Management from the University of Virginia. This is another popular choice that condenses the basics into a single 9-hour course.
        For specialized applications of project management, you may be interested in IBM’s IT Project Manager Professional Certificate, Columbia University’s Construction Management Specialization, or IBM’s DevOps and Software Engineering Professional Certificate. IT, construction, and software development are all in-demand career paths that tend to incorporate project management tools and techniques into their workflow.
        Let us know any other courses you’ve taken to grow your project management skills in the comments. Next week, we’ll continue this series with another high-income skill, user experience.
        See you then!"""
        ,"""Hello!
        We are really excited to share this next series with you, in large part because we saw your excitement with our previous issue on job skills, and because understanding how to curate an impressive portfolio of job skills is one of the most important aspects of driving your career development.
        Over the next several issues, we are going to be discussing high-income skills.
        High-income skills are skills that tend to be associated with high-paying roles and are in high demand among employers. These skills also tend to be valuable across industries and can be used productively across a wide range of job functions.
        We developed our list of high-income skills by combing several job skills studies (from organizations including the World Economic Forum, Pearson, Future Learn, Coursera, and others) and identifying the skills that came up most frequently.
        This week, we’re going to talk about data analysis skills.
        Data analysis skills are not only valuable for data analysts. Even basic data analysis skills can enhance your daily workflow. Whether you realize it or not, you are receiving and interpreting data all the time. The first lecture video in Google’s Data Analytics Professional Certificate explains this really nicely. (We recommend watching the entire video, but if you’re pressed for time, skip to the 1:45 mark to learn how data may show up in your daily life.)
        With strong data analysis skills, you can be more intentional with the data you receive, the way you interpret it, and the way you share it. Here are three examples of how data analysis skills may show up in your career, job search, or even personal life:
        1. Data analysis for highlighting accomplishments on your resume
        Supporting your resume bullet points with data can be an effective way to demonstrate your accomplishments to potential employers. Let’s examine how we can use data analysis to improve the following resume point:
        “Successfully launched an advertising campaign that increased website traffic.”
        To measure the success of your ad, you might start by recording the average number of website visitors before it was launched. Next, you could collect the average number of website visitors after it was launched. By calculating the percent of increase, you’ll have a compelling statistic to enhance your point:
        “Launched an advertising campaign that increased website traffic by 103%.”
        2. Data visualization for effective idea pitching
        Numbers can help your audience paint a mental picture. At times, an actual picture can be even better. Data visualization skills enable you to represent information in a visually appealing, easily digestible way.
        Using the example above, you might translate your last ad campaign’s 103% increase in website traffic into a line graph with a steep, eye-catching incline. To pitch an idea for a similar campaign, you might even use predictive analytics to forecast future success based on past successes.
        3. Data interpretation for expert insights about anything
        The ability to recognize patterns in data is a core aspect of data analysis. When you interpret data, you can use it to make data-informed decisions, which can ultimately lead you to results that are more likely to align with your desired outcomes. For example, interpreting data about high-income skills enabled us to develop this list, which you can use to move toward your desired career outcomes.
        Where to begin
        If you’re new to data analysis, a good place to start is with the first course in the Google Data Analytics Professional Certificate, Foundations: Data, Data, Everywhere. This course is a nice introduction to data analytics, and if you enjoy it, you can continue advancing through the rest of the certificate program.
        If you’d like to learn more specific data analysis skills, here are some additional recommendations:
        Excel Skills for Business Specialization from Macquarie University: Excel is a powerful tool for data analysis when you’re working with smaller, less complex data sets. Learning Excel (or Google Sheets, which is similar) is one beginner-friendly way to enhance your analysis capabilities.
        Introduction to Structured Query Language (SQL) from the University of Michigan: Learning SQL can enable you to analyze large data sets housed in databases. In general, SQL is still considered beginner-friendly, although it may feel a bit more advanced than Excel.
        Data Analysis and Visualization Foundations Specialization from IBM: Learning data visualization can enable you to effectively share your analyses with others, thus opening up the opportunity for you to use your analytical skills to influence high-level decision-making.
        That’s all for this week. Next week, we’ll discuss another high-income skill, project management. Until then, let us know how data analysis skills can enhance your workflow in the comments."""
        ,"""Welcome back!
        After finishing up our resume series and before we dive into our next, let’s take a pause and talk about one of the more difficult periods of the job search: the in-between.
        The in-between is the collection of passive moments after you’ve done what you can do and the only thing left is to wait and maintain faith that eventually someone will see the potential in you that you know is there. There’s only so much you can control; the in-between is not one of them.
        The in-between can be a space of discomfort. The longer it lasts, the more feelings of defeat may start creeping in. You're never alone here. And you may even see updates from others in your network, sharing their own experience of the too-long search, along with a collection of supportive and empathetic comments from digital passersby.
        But while it’s nice to feel less alone, it’s never really enough, is it?
        During these times of transition, when uncertainty begins to settle in, there are ways to reignite your drive. Here are some ideas for empowering yourself when you’re feeling stuck in the in-between.
        Make something fun.
        Eventually, you are going to get a job. But right now, in the in-between, you have this unexpected open space to use your skills in whatever way you want to use them. So choose fun! Remind yourself why you pursued these skills in the first place, and how good it can feel to use them when the stakes are low.
        Here are some examples:
        Interested in social media marketing? Share your gardening hobby with the world on TikTok.
        Aspiring data analyst? Create a spreadsheet to track your TV habit.
        Going into web development? Build a website dedicated to the joys of the color yellow.
        To come up with your own idea, all you need to do is combine your skill with something else you like. It doesn’t have to be profound; it just has to feel good.
        Plus, if you like the thing you make for fun, you can incorporate it into your resume or portfolio. This can be a nice way to demonstrate workplace skills like creativity and innovation alongside your sought-after technical skills.
        Learn something completely different.
        Once you have a solid resume and adaptable cover letter template, the job application process can feel tedious. Pull yourself out of the monotony by activating a different part of your brain. Learn something new.
        There are chemical benefits to learning—if you haven’t taken Learning How to Learn yet, this is a good course to start understanding what’s happening in your brain when you learn. (And it’s free.) But more importantly, those brain benefits can activate mood benefits.
        Learning feels good. It’s a confidence builder and it’s a way to gain a new perspective on the world around you. When you learn something new in one subject, you’ll likely start making new connections in completely different areas.
        Restore your brain with a wildcard course. There are plenty of hidden gems on Coursera. Here are some recommendations:
        A Life of Happiness and Fulfillment from the Indian School of Business
        Dynamic Public Speaking from the University of Washington
        Psychological First Aid from Johns Hopkins University
        Modern and Contemporary Art and Design from the Museum of Modern Art
        Creative Writing from Wesleyan University
        Set boundaries.
        Looking for a new job can feel like a full-time job in itself. The difference is, when you’re job searching, there aren’t always clear milestones suggesting that you take a step back and focus on something else. There are always more job sites to check out, more applications to send, and more people to network with. There’s also more life to live outside of your job search.
        The tenets of maintaining a healthful work-life balance hold true even when your work is to find work, and setting boundaries to guide your job search is one way you can invite more balance into your daily routine.
        Your boundaries can be anything that feel right for you. Perhaps you want to send a certain number of job applications per day, schedule one networking meeting every week, or stop checking your email after 7 p.m. every day. You may experience some trial and error as you figure out your boundaries, but it tends to be worth the effort.
        This feels like a good time to mention The Science of Well-Being from Yale University. Another popular free course, this one explores happiness and how to achieve it from a psychological standpoint.
        That’s all for this week. If you’re struggling with the in-between, we’re rooting for you. If you have tips for staying positive, please share them in the comments. Next week, we’re going to start our series on high-income skills and explore how you can incorporate them into your professional toolkit. See you then."""

    ]
category = [
        "Career Advice",
        "Technology",
        "Finance",
        "Career Advice",
        "Technology",
        "Work-Life Balance",
        "Finance",
        "Technology",
        "Technology",
        "Entrepreneurship",
        "Technology",
        "Finance",
        "Personal Development",
        "E-commerce",
        "Career Advice",
        "Cybersecurity",
        "Career Advice",
        "Finance",
        "Project Management",
        "Technology",
        "SocialMediaMarketing",
        "Finance",
        "Technology",
        "Career Advice",
        "Technology",
        "Finance",
        "Technology",
        "Machine Learning",
        "Acheivement",
        "UserInterface",
        "product Manag",
        "DataAnalysis",
        "Student_related"
    ]
print(len(post_text),len(category))
data = {
    'Post_Text': np.array(post_text),
    'Category':np.array(category)
}
df = pd.DataFrame(data)
df.head()

33 33


,Post_Text,Category
0,Exciting news! I just discovered 5 valuable t...,Career Advice
1,Artificial Intelligence is reshaping the worl...,Technology
2,Are you interested in investing in cryptocurr...,Finance
3,Building a strong personal brand is key to ca...,Career Advice
4,"As an entrepreneur, productivity is crucial t...",Technology


In [148]:
df['Post_Text'] = df['Post_Text'].apply(lambda x: TextPreprocessing().preprocess_text(x))
df['Post_Text'] = df['Post_Text'].apply(lambda x: ' '.join(x))
df.head()

,Post_Text,Category
0,exciting news discovered valuable tip signific...,Career Advice
1,artificial intelligence reshaping world know l...,Technology
2,interested investing cryptocurrency look ive p...,Finance
3,building strong personal brand key career succ...,Career Advice
4,entrepreneur productivity crucial success post...,Technology


In [111]:
# df['Post_Text'][32]

In [112]:
set(map(len, df['Post_Text']))

{24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 48, 74, 163, 345, 374, 425, 457}

In [113]:
!pip install gensim
!pip install top2vec

In [114]:
import numpy as np

# Example array with varying row lengths
array = np.array(df['Post_Text'].to_list())

# Find the maximum length in the column
max_length = max(len(row) for row in array)

# Iterate over each row and increase its size to the maximum length
for i, row in enumerate(array):
    if len(row) < max_length:
        padding = ['<PAD>'] * (max_length - len(row))
        array[i] = np.concatenate([row, padding])

print(set(map(len, array)))

{457}


<ipython-input-114-37c5b932e33c>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.array(df['Post_Text'].to_list())


In [115]:
df['padded_text']= pd.DataFrame(array)
df.head()

,Post_Text,Category,padded_text
0,"[exciting, news, discovered, valuable, tip, si...",Career Advice,"[exciting, news, discovered, valuable, tip, si..."
1,"[artificial, intelligence, reshaping, world, k...",Technology,"[artificial, intelligence, reshaping, world, k..."
2,"[interested, investing, cryptocurrency, look, ...",Finance,"[interested, investing, cryptocurrency, look, ..."
3,"[building, strong, personal, brand, key, caree...",Career Advice,"[building, strong, personal, brand, key, caree..."
4,"[entrepreneur, productivity, crucial, success,...",Technology,"[entrepreneur, productivity, crucial, success,..."


In [116]:
df['padded_text'] = df['padded_text'].apply(lambda x: ' '.join(x))
df.head()

,Post_Text,Category,padded_text
0,"[exciting, news, discovered, valuable, tip, si...",Career Advice,exciting news discovered valuable tip signific...
1,"[artificial, intelligence, reshaping, world, k...",Technology,artificial intelligence reshaping world know l...
2,"[interested, investing, cryptocurrency, look, ...",Finance,interested investing cryptocurrency look ive p...
3,"[building, strong, personal, brand, key, caree...",Career Advice,building strong personal brand key career succ...
4,"[entrepreneur, productivity, crucial, success,...",Technology,entrepreneur productivity crucial success post...


In [82]:
import pandas as pd

User_data = {
    'UserID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'UserName': ['JohnDoe', 'JaneSmith', 'MikeJohnson', 'EmilyBrown', 'DavidWilson',
                 'SarahDavis', 'MichaelLee', 'JenniferTaylor', 'RobertAnderson', 'OliviaMoore'],
    'Brief intro': ['Experienced software engineer specializing in web development.',
                    'Passionate marketing professional with a focus on social media.',
                    'Financial analyst with expertise in risk management.',
                    'Creative graphic designer with a love for minimalist design.',
                    'Veterinarian dedicated to providing quality care for animals.',
                    'Experienced teacher specializing in elementary education.',
                    'Data scientist with a background in machine learning.',
                    'Talented photographer capturing moments with a unique perspective.',
                    'Experienced lawyer specializing in corporate law.',
                    'Health and wellness coach promoting a balanced lifestyle.'],
    'Area of interest': ['Technology', 'Marketing', 'Finance', 'Design', 'Veterinary',
                         'Education', 'Data Science', 'Photography', 'Law', 'Health & Wellness']
}

User_df = pd.DataFrame(User_data)
User_df.head()

,UserID,UserName,Brief intro,Area of interest
0,1,JohnDoe,Experienced software engineer specializing in ...,Technology
1,2,JaneSmith,Passionate marketing professional with a focus...,Marketing
2,3,MikeJohnson,Financial analyst with expertise in risk manag...,Finance
3,4,EmilyBrown,Creative graphic designer with a love for mini...,Design
4,5,DavidWilson,Veterinarian dedicated to providing quality ca...,Veterinary


In [121]:
User_df['Prof_data'] = User_df['UserName']+User_df['Brief intro']+User_df['Area of interest']

In [95]:
df.head()

,Post_Text,Category,padded_text
0,"[exciting, news, discovered, valuable, tip, si...",Career Advice,exciting news discovered valuable tip signific...
1,"[artificial, intelligence, reshaping, world, k...",Technology,artificial intelligence reshaping world know l...
2,"[interested, investing, cryptocurrency, look, ...",Finance,interested investing cryptocurrency look ive p...
3,"[building, strong, personal, brand, key, caree...",Career Advice,building strong personal brand key career succ...
4,"[entrepreneur, productivity, crucial, success,...",Technology,entrepreneur productivity crucial success post...


In [149]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Preprocess the text data (remove stopwords, convert to lowercase, etc.)
preprocessed_docs = df['Post_Text']

# Tokenize the preprocessed text
tokenized_docs = preprocessed_docs.apply(word_tokenize)

# Create a dictionary from the tokenized text
dictionary = Dictionary(tokenized_docs)

# Convert tokenized text to numerical representation (BOW or TF-IDF)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

# Train the LDA model
lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)

# Get the topic distribution for each document
document_topics = lda_model.get_document_topics(corpus)

# Iterate over the document topics and print the topic for each document
for i, topics in enumerate(document_topics):

  print(f"Post {i+1} - Topic Distribution: {topics}")


Post 1 - Topic Distribution: [(3, 0.96399516)]
Post 2 - Topic Distribution: [(1, 0.96785116)]
Post 3 - Topic Distribution: [(3, 0.9699957)]
Post 4 - Topic Distribution: [(0, 0.9756708)]
Post 5 - Topic Distribution: [(4, 0.40403455), (7, 0.56929153)]
Post 6 - Topic Distribution: [(8, 0.9709653)]
Post 7 - Topic Distribution: [(5, 0.97187227)]
Post 8 - Topic Distribution: [(1, 0.9718712)]
Post 9 - Topic Distribution: [(2, 0.9742827)]
Post 10 - Topic Distribution: [(5, 0.96896154)]
Post 11 - Topic Distribution: [(4, 0.96999484)]
Post 12 - Topic Distribution: [(5, 0.39907497), (8, 0.57739234)]
Post 13 - Topic Distribution: [(8, 0.9699945)]
Post 14 - Topic Distribution: [(0, 0.96785396)]
Post 15 - Topic Distribution: [(2, 0.9699952)]
Post 16 - Topic Distribution: [(8, 0.97271705)]
Post 17 - Topic Distribution: [(6, 0.9718705)]
Post 18 - Topic Distribution: [(5, 0.24291201), (6, 0.7328421)]
Post 19 - Topic Distribution: [(2, 0.5727488), (4, 0.39965832)]
Post 20 - Topic Distribution: [(0, 0.52

In [151]:
from sklearn.metrics.pairwise import cosine_similarity, sigmoid_kernel

for user in User_df['Prof_data']:
    text = user
    preprocessed_words = TextPreprocessing().preprocess_text(text)

    # Get the topic distribution for the preprocessed words
    topic_dist_words = lda_model.get_document_topics(dictionary.doc2bow(preprocessed_words))

    similarities = []
    Sig_similarities = []
    for post in df['Post_Text']:
        preprocessed_post = TextPreprocessing().preprocess_text(post)  # Preprocess the post text
        bow_post = dictionary.doc2bow(preprocessed_post)  # Convert the preprocessed post to BoW format
        topic_dist_post = lda_model.get_document_topics(bow_post)

        topic_dist_words_values = [val for _, val in topic_dist_words]  # Extract topic distribution values for words
        topic_dist_post_values = [0.0] * lda_model.num_topics  # Initialize topic distribution values for the post

        for topic_id, prob in topic_dist_post:
            topic_dist_post_values[topic_id] = prob  # Assign the probability to the corresponding topic

        similarity = cosine_similarity([topic_dist_words_values], [topic_dist_post_values])
        Sig_similarity = sigmoid_kernel([topic_dist_words_values], [topic_dist_post_values])

        similarities.append(similarity[0][0])  # Append the similarity value to the list
        Sig_similarities.append(Sig_similarity[0][0])  # Append the similarity value to the list

    df['similarity'] = similarities  # Add the similarity values to the dataframe
    print("Cosine Function")
    # Sort the posts based on their similarity
    recommendations = [(similarity, index, post) for similarity, index, post in sorted(zip(similarities, df.index, df['Post_Text']), reverse=True)]
    index_id = []
    # Print the recommended posts
    for similarity, index, post in recommendations:
        index_id.append(index)
        print()
    print(index_id)

    print("Sigmoid Kernel Function")

    # Sort the posts based on their similarity
    Sig_recommendations = [(similarity, index, post) for similarity, index, post in sorted(zip(Sig_similarities, df.index, df['Post_Text']), reverse=True)]
    sig_ind = []
    # Print the recommended posts
    for similarity, index, post in Sig_recommendations:
      sig_ind.append(index)
    print(sig_ind)

Cosine Function
[29, 27, 20, 13, 3, 28, 26, 15, 12, 5, 19, 11, 32, 4, 18, 17, 22, 10, 30, 2, 0, 31, 14, 8, 7, 1, 25, 24, 23, 21, 9, 6, 16]
Sigmoid Kernel Function
[29, 27, 3, 20, 13, 28, 26, 15, 5, 12, 19, 11, 32, 31, 30, 17, 8, 25, 4, 18, 22, 7, 6, 16, 10, 2, 14, 21, 9, 23, 1, 24, 0]
Cosine Function
[29, 27, 20, 13, 3, 19, 32, 4, 18, 11, 17, 30, 2, 0, 31, 14, 8, 25, 24, 23, 21, 9, 6, 7, 1, 16, 22, 10, 28, 26, 15, 12, 5]
Sigmoid Kernel Function
[29, 27, 3, 20, 13, 19, 32, 31, 30, 28, 26, 11, 17, 8, 25, 4, 15, 18, 6, 7, 16, 22, 5, 2, 14, 10, 12, 21, 9, 23, 1, 24, 0]
Cosine Function
[32, 30, 2, 0, 19, 4, 18, 11, 17, 25, 24, 23, 21, 9, 6, 16, 29, 27, 20, 13, 3, 7, 1, 31, 14, 8, 22, 10, 28, 26, 15, 12, 5]
Sigmoid Kernel Function
[30, 2, 0, 32, 31, 29, 28, 27, 26, 11, 17, 19, 3, 8, 25, 4, 15, 18, 6, 16, 20, 7, 22, 5, 14, 10, 12, 21, 9, 23, 13, 1, 24]
Cosine Function
[29, 27, 20, 13, 3, 19, 32, 4, 18, 11, 17, 31, 14, 8, 30, 2, 0, 22, 10, 16, 7, 1, 28, 26, 15, 12, 5, 25, 24, 23, 21, 9, 6]
Sig